<a href="https://colab.research.google.com/github/romlingroup/flatpack-ai/blob/main/notebooks/flatpack_ai_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# flatpack.ai - RNN
This Colab notebook demonstrates how to train a Recurrent Neural Network (RNN) on a text dataset and then use the trained model to generate new text sequences. The model is trained character by character, learning the patterns and structure of the input text, and can then generate coherent and contextually relevant text sequences.

https://github.com/romlingroup/flatpack-ai

In [ ]:
!pip uninstall flatpack -y
!pip install flatpack

In [ ]:
import torch
from flatpack import datasets, instructions, models

text = datasets.download_text("https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt")[:100000]
chars = sorted(set(text))
indexed_text = [chars.index(char) for char in text]
char_to_index = {char: i for i, char in enumerate(chars)}
index_to_char = {i: char for i, char in enumerate(chars)}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

instructions.build(
    batch_size=128,
    char_to_index=char_to_index,
    device=device,
    epochs=10,
    framework='pytorch',
    index_to_char=index_to_char,
    model_type='rnn',
    save_dir='/content/saved_model',
    user_train_function=lambda epochs, batch_size: models.RNN.train_model(
        batch_size=batch_size,
        dataset=datasets.TextDataset(indexed_text, seq_length=64),
        device=device,
        embed_size=32,
        epochs=epochs,
        hidden_size=128,
        num_layers=4,
        vocab_size=len(chars),
    )
)

In [ ]:
import random
import torch
from flatpack import models

random.seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.RNN(32, 128, 4).to(device)
model.load_vocab_size('/content/saved_model')
model.load_state_dict(models.RNN.load_torch_model('/content/saved_model/rnn_model.pth'))
model.embedding, model.rnn, model.fc = model.embedding.to(device), model.rnn.to(device), model.fc.to(device)

print("Generated text:", model.generate_text('/content/saved_model', start_sequence="To be, or not to be", generate_length=1024, temperature=1.0, device=device))